This notebook contains the code for Ising model animations used in [Brain Criticality video](https://youtu.be/vwLb3XlPCB4)

In [ ]:
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import time

from numba import jit
from PIL import Image
from matplotlib import colors

from matplotlib.animation import FuncAnimation
from tqdm.notebook import tqdm

import napari


import sys
sys.path.append("../../../Animation/")
from AK_animation_utils import *

## Simulation code

Taken from https://github.com/rytheranderson/fast-python-ising-model

In [ ]:
@jit
def initialize_lattice_random(nrow, ncol):
    return np.where(np.random.random((nrow, ncol)) > 0.5, 1, -1)


@jit
def initialize_lattice_uniform(nrow, ncol, spin=1):
    return np.ones((nrow, ncol), dtype=np.int64) * spin


@jit
def system_energy(lattice, J, H):
    """
        J is the spin interaction parameter,
        J > 0 = ferromagnetic
        J < 0 = antiferromagnetic
        H is an external magnetic field (constant)
    """
    nrow, ncol = lattice.shape
    E = 0.0

    for i in range(nrow):
        for j in range(ncol):

            S = lattice[i, j]
            NS = (lattice[(i+1) % nrow, j] + 
                  lattice[i, (j+1) % ncol] + 
                  lattice[(i-1) % nrow, j] + 
                  lattice[i, (j-1) % ncol])
            E += -1 * ((J * S * NS) + (H * S))
    return E/4


@jit
def system_magnetization(lattice):
    return np.sum(lattice)


@jit
def mc_cycle(lattice, J, H, T):
    """
        A single MC cycle (considering all lattice points)
        T is the temperature
    """
    T = float(T)
    naccept = 0 
    nrow, ncol = lattice.shape 
    E = system_energy(lattice, J, H) 
    M = system_magnetization(lattice)

    for i in range(nrow): 
        for j in range(ncol):

            S = lattice[i, j]
            NS = (lattice[(i+1) % nrow, j] +
                  lattice[i, (j+1) % ncol] +
                  lattice[(i-1) % nrow, j] +
                  lattice[i, (j-1) % ncol])
            dE = 2*J*S*NS + 2*H*S
            accept = np.random.random()

            if dE < 0.0 or accept < np.exp((-1.0 * dE)/T):
                naccept += 1
                S *= -1
                E += dE
                M += 2*S
            lattice[i, j] = S
    return lattice, E, M, naccept


@jit
def run(lattice, n_cycles, J=1, H=0, T=1.0, standard_output=False):
    nrow, ncol = lattice.shape

    lattice_evolve = [np.zeros((nrow, ncol)) for i in range(n_cycles)]
    energy_vs_step = []
    magnet_vs_step = []

    for cyc in range(n_cycles):

        if standard_output:
            print(f'cycle {cyc + 1} out of {n_cycles}')

        lattice, E, M, naccept = mc_cycle(lattice, J, H, T)
        lattice_evolve[cyc] += lattice
        energy_vs_step.append(E)
        magnet_vs_step.append(M)

    return lattice, energy_vs_step, magnet_vs_step, lattice_evolve

def cooling(lattice, temp_range, n_cycles, J=1, H=0):
    """ a series of runs at decreasing temperatures """

    summary = []
    frames = []
    FL = lattice
    for T in temp_range:
        print(f'Temperature = {np.round(T,3)}')
        FL, EvS, MvS, LvS = run(lattice, n_cycles, J=J, H=H, T=T)
        summary.append([J, EvS, MvS])
        frames.extend(LvS)

    return summary, frames

def continuous_cooling(lattice, temperature_values, J=1, H=0):
    summary = []
    frames = []
    FL = lattice

    for T in temperature_values:
        FL, EvS, MvS, LvS = run(FL, 1, J=J, H=H, T=T)
        summary.append([J, EvS, MvS])
        frames.extend(LvS)
    return summary, frames

## Animation functions

In [ ]:
def animate_lattice_pcolormesh(lattice_evolution_array):
    '''Animage Ising model as pcolormesh (squares with black spacing between them)'''
    
    red_blue_cmap = get_continuous_cmap(["#FF705E","#50BCE3"])
    fig, ax = plt.subplots(1,1,figsize=(12,12),dpi=300)
    ax.set_facecolor("black")
    fig.set_facecolor("black")
    cmesh = ax.pcolormesh(lattice_evolution_array[0,:,:], edgecolors='k', vmin=-1, vmax=1,linewidth=2, cmap=red_blue_cmap)

    def animate(frame_num):
        cmesh.set_array(lattice_evolution_array[frame_num,:,:])
        return cmesh,

    anim = FuncAnimation(fig, animate,frames=tqdm(range(lattice_evolution_array.shape[0])), interval=100)
    return anim
    
def animate_lattice_imshow(lattice_evolution_array):
    '''Animage Ising model as imshow (pixel grid with no spacing)'''
    
    red_blue_cmap = get_continuous_cmap(["#FF705E","#50BCE3"])

    fig, ax = plt.subplots(1,1,figsize=(12,12),dpi=300)
    ax.set_facecolor("black")
    fig.set_facecolor("black")
    cmesh = ax.imshow(lattice_evolution_array[0,:,:], vmin=-1, vmax=1,cmap=red_blue_cmap)

    def animate(frame_num):
        cmesh.set_data(lattice_evolution_array[frame_num,:,:])
        return cmesh,

    anim = FuncAnimation(fig, animate,frames=tqdm(range(lattice_evolution_array.shape[0])), interval=100)
    return anim

## Cold and hot lattices

In [ ]:
# Cold
lattice_cold = initialize_lattice_random(75,75)
_, _, _, lattice_evolve_cold = run(lattice_cold, 2000, T=1.5)
lattice_evolve_cold = np.array(lattice_evolve_cold)[1000::]

In [ ]:
anim = animate_lattice_pcolormesh(lattice_evolve_cold)
anim.save("Ising model cold.mp4")

In [ ]:
# Hot
lattice_hot = initialize_lattice_random(75,75)
_, _, _, lattice_evolve_hot = run(lattice_hot, 2000, T=4)
lattice_evolve_hot = np.array(lattice_evolve_hot)[1000::]

In [ ]:
anim = animate_lattice_pcolormesh(lattice_evolve_hot)
anim.save("Ising model hot.mp4")

## Energy minimization scene

In [ ]:
# ---- Maximum energy (hot)
lattice = initialize_lattice_random(100,100)
_, _, _, lattice_evolve = run(lattice, 1500, T=5)
lattice_evolve_maximum_enegy = np.array(lattice_evolve[1000::])

anim = animate_lattice_pcolormesh(lattice_evolve_maximum_enegy)
plt.savefig("Maximum energy.png") # Note that here I'm saving only the still .png image

In [ ]:
# ---- Medium energy
lattice = initialize_lattice_random(100,100)
_, _, _, lattice_evolve = run(lattice, 1500, T=2.1)
lattice_evolve_medium_enegy = np.array(lattice_evolve[500::])

anim = animate_lattice_pcolormesh(lattice_evolve_medium_enegy)
plt.savefig("Medium energy.png")

In [ ]:
# ---- Minimum energy (cold)
lattice = initialize_lattice_random(100,100)
_, _, _, lattice_evolve = run(lattice, 1500, T=1.5)
lattice_evolve_minimum_enegy = np.array(lattice_evolve[1000::])

anim = animate_lattice_pcolormesh(lattice_evolve_minimum_enegy)
plt.savefig("Minimum energy.png")

## Heating animation

In [ ]:
lattice = initialize_lattice_uniform(200,200)
temperature_values = np.linspace(0.5, 6, 500) # Note that this from T=0.5 to T=6, which heating, rather than cooling :) 
summary, frames = continuous_cooling(lattice, temperature_values)
frames = np.array(frames)

anim = animate_lattice_imshow(frames)
anim.save("Ising model heating.mp4")

## Boltzmann distribution

In [ ]:
from scipy.stats import maxwell
import matplotlib.cm as cm

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,7),dpi=300)
fig.set_facecolor("black")
ax.set_facecolor("black")
ax.axis(False)
x = np.linspace(0,10,1000)
temps = np.linspace(0.2, 4, 20)

def boltzmann_prob(e,T):
    return np.exp(-1*e/T)

for k,T in enumerate(temps):
    ax.plot(x,boltzmann_prob(x,T), color=cm.cool(k/(len(temps)-1)),lw=2.5)
    
plt.savefig("Boltzmann dist.svg")

## Cluster sizes

In [ ]:
import scipy.ndimage as ndi

In [ ]:
lattice = initialize_lattice_random(200,200)
lattice, _, _, lattice_evolve = run(lattice,1000,T=2.5)

labels, nlab = ndi.label(lattice+1) # Finding clusters
label_sizes = [(np.sum(labels==k),k) for k in range(1,nlab)] # Cluster sizes
label_sizes = sorted(label_sizes,reverse=True)

In [ ]:
def plot_cluster(labels, cluster_id="all"):
    fig, ax = plt.subplots(1,1,figsize=(12,12),dpi=300)
    ax.axis(False)
    ax.set_facecolor("black")
    fig.set_facecolor("black")
    if cluster_id=="all":
        ax.imshow(labels, cmap="binary_r", vmin=0, vmax=1)
    else:
        ax.imshow(labels==cluster_id, cmap="binary_r", vmin=0, vmax=1)

In [ ]:
larger_clusters_k = [tup[1] for tup in label_sizes[0:3]]
smaller_clusters_k = [tup[1] for tup in label_sizes[5:8]]

In [ ]:
for k in ["all"] + larger_clusters_k+smaller_clusters_k:
    plot_cluster(labels, k)
    plt.savefig(f"Cluster {k}.png",bbox_inches="tight")

## Scale-free properties on graphs

In [ ]:
def f(x):
    return x**(-2.3) # Power law
def g(x):
    return 10*np.exp(-2*x) # Exponential

In [ ]:
def setup_figure():
    ''' Setting up figure and axis'''
    fig, ax = plt.subplots(1,1, figsize=(15,9), dpi=300)
    ax.set_facecolor("black")
    fig.set_facecolor("black")
    ax.tick_params(colors="white", labelsize=15)
    ax.spines["bottom"].set_color("white")
    ax.spines["left"].set_color("white")
    return fig, ax

def adjust_axis_lims(x_range, function):
    ''' Adjust the axis limits (used in animation)'''
    ax.set_xlim(x_range[0],x_range[1])
    ax.set_ylim(function(x_range[1]), function(x_range[0]))

### Power law function

In [ ]:
fig, ax = setup_figure()
x = np.linspace(1,200,5000)
line, = ax.plot(x,f(x),color="salmon", lw=7)
adjust_axis_lims((1,2), f)

def animate_f_graph(frame):
    start, end = frame, 2*frame
    adjust_axis_lims((start, end),f)
    return ax,

anim = FuncAnimation(fig, animate_f_graph, frames=tqdm(np.linspace(1, 50, 1000)), interval=40)
anim.save("Power law graph.mp4")

### Exponential function

In [ ]:
fig, ax = setup_figure()
x = np.linspace(1,200,5000)
line, = ax.plot(x,g(x),color="#03e3fc", lw=7)
adjust_axis_lims((1,2), g)

def animate_g_graph(frame):
    start, end = frame, 2*frame
    adjust_axis_lims((start, end),g)
    return ax,

anim = FuncAnimation(fig, animate_g_graph, frames=tqdm(np.linspace(1, 10, 1000)), interval=40)
anim.save("Exponential graph.mp4")